In [46]:
import pandas as pd
from pymongo import MongoClient
import re
import csv


### Connection avec mongodb


In [47]:
client = MongoClient("localhost:27017")
database = client.scrapy
movies = database.movies
serie = database.serie
movies.drop()
serie.drop()

### Nettoyage des données

In [48]:
df_movie = pd.read_csv("csv/items.csv")
# df_serie = pd.read_csv("csv/serie.csv")

In [49]:
df_movie["titre_original"].replace(to_replace="Original title: ", value='', regex=True,inplace=True)


In [50]:
df_movie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
# df_serie["duree"].replace(to_replace=",", value='', regex=True,inplace=True)
def convertion_duree(s):
    
    split_cell = re.split("\s", s)
    duree = 0
    for elt in split_cell:
        if "h" in elt:
            duree = int(re.split("h", elt)[0]) *60 +duree
        elif "m" in elt:
            duree =int(re.split("m", elt)[0]) +duree
    return duree
    




# df_movie['duree'] = df_movie['duree'].apply(lambda x :  convertion_duree(str(x)))
# df_serie['duree'] = df_serie['duree'].apply(lambda x :  convertion_duree(str(x)))



In [51]:
df_movie.to_csv("csv/items.csv",index=False)
# df_serie.to_csv("csv/serie.csv",index=False)

### Import des données dans mongodb

In [52]:
header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original","url"]
csvfile = open('csv/items.csv', 'r')
reader = csv.DictReader( csvfile )

for each in reader:
    row={}
    for field in header:

        if field in  ["acteurs","genre","pays"]:
            row[field]=each[field].split(",")
        elif field in ["date","duree"]:
            row[field]=int(each[field])
        elif field =="score":
            row[field]=float(each[field])
        else :
            row[field]=each[field]
        
    movies.insert_one(row)

In [53]:
header = ["acteurs","date","descriptions","duree","genre","pays","public","score","titre","titre_original"]
csvfile = open('csv/serie.csv', 'r')
reader = csv.DictReader( csvfile )

for each in reader:
    row={}
    for field in header:

        if field in  ["acteurs","genre","pays"]:
            row[field]=each[field].split(",")
        elif field in ["duree"]:
            row[field]=int(each[field])
        elif field =="score":
            row[field]=float(each[field])
        else :
            row[field]=each[field]  
        
    serie.insert_one(row)

## question n°1

In [54]:
movies.find().sort("duree",-1).limit(1)[0]

{'_id': ObjectId('639afa454158f6a45ece754b'),
 'acteurs': ['Clark Gable', 'Vivien Leigh', 'Thomas Mitchell'],
 'date': 1939,
 'descriptions': 'The manipulative daughter of a Georgia plantation owner conducts a turbulent romance with a roguish profiteer during the American Civil War and Reconstruction periods.',
 'duree': 238,
 'genre': ['Drama', 'Romance', 'War'],
 'pays': ['United States'],
 'public': 'Passed',
 'score': 8.2,
 'titre': 'Gone with the Wind',
 'titre_original': '',
 'url': 'https://m.media-amazon.com/images/M/MV5BYjUyZWZkM2UtMzYxYy00ZmQ3LWFmZTQtOGE2YjBkNjA3YWZlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_QL75_UX190_CR0,5,190,281_.jpg'}

## Question n°2

In [55]:
five_movies=movies.find().sort("score",-1).limit(5)
for movie in five_movies:
    print(movie)

{'_id': ObjectId('639afa454158f6a45ece74e0'), 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'date': 1994, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'duree': 142, 'genre': ['Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.3, 'titre': 'The Shawshank Redemption', 'titre_original': '', 'url': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_QL75_UX190_CR0,0,190,281_.jpg'}
{'_id': ObjectId('639afa454158f6a45ece75d8'), 'acteurs': ['Marlon Brando', 'Al Pacino', 'James Caan'], 'date': 1972, 'descriptions': 'The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son.', 'duree': 175, 'genre': ['Crime', 'Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.2, 'titre': 'The Godfather', 'titre_original': '',

## Question n°3


In [56]:
Freeman_movies = movies.find({"acteurs":{"$in":["Morgan Freeman"]}})
for movie in Freeman_movies:
    print(movie)

{'_id': ObjectId('639afa454158f6a45ece74e0'), 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'date': 1994, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'duree': 142, 'genre': ['Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.3, 'titre': 'The Shawshank Redemption', 'titre_original': '', 'url': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_QL75_UX190_CR0,0,190,281_.jpg'}
{'_id': ObjectId('639afa454158f6a45ece74ea'), 'acteurs': ['Morgan Freeman', 'Brad Pitt', 'Kevin Spacey'], 'date': 1995, 'descriptions': 'Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.', 'duree': 127, 'genre': ['Crime', 'Drama', 'Mystery'], 'pays': ['United States'], 'public': 'R', 'score': 8.6, 'titre': 'Se7en', 'titre_original': '', 'url': 'https://m.media-amazon.com/

In [57]:
Cruise_movies = movies.find({"acteurs":{"$in":["Tom Cruise"]}})
for movie in Cruise_movies:
    print(movie)

{'_id': ObjectId('639afa454158f6a45ece759a'), 'acteurs': ['Tom Cruise', 'Jennifer Connelly', 'Miles Teller'], 'date': 2022, 'descriptions': "After thirty years, Maverick is still pushing the envelope as a top naval aviator, but must confront ghosts of his past when he leads TOP GUN's elite graduates on a mission that demands the ultimate sacrifice from those chosen to fly it.", 'duree': 130, 'genre': ['Action', 'Drama'], 'pays': ['United States'], 'public': 'PG-13', 'score': 8.4, 'titre': 'Top Gun: Maverick', 'titre_original': '', 'url': 'https://m.media-amazon.com/images/M/MV5BZWYzOGEwNTgtNWU3NS00ZTQ0LWJkODUtMmVhMjIwMjA1ZmQwXkEyXkFqcGdeQXVyMjkwOTAyMDU@._V1_QL75_UX190_CR0,0,190,281_.jpg'}


## Question n°4

In [58]:
horror_movies = movies.find({"genre":{"$in":["Horror"]}}).sort("score",-1).limit(3)
for movie in horror_movies:
    print(movie)

{'_id': ObjectId('639afa454158f6a45ece75c9'), 'acteurs': ['Anthony Perkins', 'Janet Leigh', 'Vera Miles'], 'date': 1960, 'descriptions': "A Phoenix secretary embezzles $40,000 from her employer's client, goes on the run and checks into a remote motel run by a young man under the domination of his mother.", 'duree': 109, 'genre': ['Horror', 'Mystery', 'Thriller'], 'pays': ['United States'], 'public': 'R', 'score': 8.5, 'titre': 'Psycho', 'titre_original': '', 'url': 'https://m.media-amazon.com/images/M/MV5BNTQwNDM1YzItNDAxZC00NWY2LTk0M2UtNDIwNWI5OGUyNWUxXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_QL75_UX190_CR0,0,190,281_.jpg'}
{'_id': ObjectId('639afa454158f6a45ece75b7'), 'acteurs': ['Sigourney Weaver', 'Tom Skerritt', 'John Hurt'], 'date': 1979, 'descriptions': 'The crew of a commercial spacecraft encounter a deadly lifeform after investigating an unknown transmission.', 'duree': 117, 'genre': ['Horror', 'Sci-Fi'], 'pays': ['United Kingdom', 'United States'], 'public': 'R', 'score': 8.5, 'titre'

In [59]:
drama_movies = movies.find({"genre":{"$in":["Drama"]}}).sort("score",-1).limit(3)
for movie in drama_movies:
    print(movie)

{'_id': ObjectId('639afa454158f6a45ece74e0'), 'acteurs': ['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'], 'date': 1994, 'descriptions': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.', 'duree': 142, 'genre': ['Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.3, 'titre': 'The Shawshank Redemption', 'titre_original': '', 'url': 'https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_QL75_UX190_CR0,0,190,281_.jpg'}
{'_id': ObjectId('639afa454158f6a45ece75d8'), 'acteurs': ['Marlon Brando', 'Al Pacino', 'James Caan'], 'date': 1972, 'descriptions': 'The aging patriarch of an organized crime dynasty in postwar New York City transfers control of his clandestine empire to his reluctant youngest son.', 'duree': 175, 'genre': ['Crime', 'Drama'], 'pays': ['United States'], 'public': 'R', 'score': 9.2, 'titre': 'The Godfather', 'titre_original': '',

In [60]:
comedy_movies = movies.find({"genre":{"$in":["Comedy"]}}).sort("score",-1).limit(3)
for movie in comedy_movies:
    print(movie)

{'_id': ObjectId('639afa454158f6a45ece74e6'), 'acteurs': ['Roberto Benigni', 'Nicoletta Braschi', 'Giorgio Cantarini'], 'date': 1997, 'descriptions': 'When an open-minded Jewish waiter and his son become victims of the Holocaust, he uses a perfect mixture of will, humor, and imagination to protect his son from the dangers around their camp.', 'duree': 116, 'genre': ['Comedy', 'Drama', 'Romance'], 'pays': ['Italy'], 'public': 'PG-13', 'score': 8.6, 'titre': 'Life Is Beautiful', 'titre_original': 'La vita è bella', 'url': 'https://m.media-amazon.com/images/M/MV5BYmJmM2Q4NmMtYThmNC00ZjRlLWEyZmItZTIwOTBlZDQ3NTQ1XkEyXkFqcGdeQXVyMTQxNzMzNDI@._V1_QL75_UX190_CR0,1,190,281_.jpg'}
{'_id': ObjectId('639afa454158f6a45ece75b5'), 'acteurs': ['Charles Chaplin', 'Virginia Cherrill', 'Florence Lee'], 'date': 1931, 'descriptions': '', 'duree': 87, 'genre': ['Comedy', 'Drama', 'Romance'], 'pays': ['United States'], 'public': 'G', 'score': 8.5, 'titre': 'City Lights', 'titre_original': '', 'url': ''}
{'_i

## Question 6

In [61]:
american_movies = movies.find().sort("score",-1).limit(100)
count=0
for movie in american_movies:
    if "United States" in movie["pays"]:
        count +=1
pourcentage = count/250
print(pourcentage*100)

29.599999999999998


In [62]:
french_movies = movies.find({"pays":{"$in":["France"]}}).sort("score",-1).limit(100)
for movie in french_movies:
    print(movie["titre"],movie["score"])

City of God 8.6
Cinema Paradiso 8.5
The Intouchables 8.5
Léon: The Professional 8.5
The Pianist 8.5
Capernaum 8.4
The Boat 8.4
Amadeus 8.4
The Lives of Others 8.4
A Separation 8.3
Incendies 8.3
Amélie 8.3
The Passion of Joan of Arc 8.2
The Wages of Fear 8.2
Casino 8.2
Ran 8.2
The Father 8.2
The 400 Blows 8.1
La haine 8.1
Before Sunset 8.1
Wild Tales 8.1


In [63]:
top_100 = movies.find().sort("score", -1)[:100]
france = 0
us = 0
for i in top_100:
    if "France" in i['pays']  :
        france += 1
    elif "United States" in i['pays'] :
        us += 1
f"Dans le Top 100 des films : US ({us}%) FR ({france}%)"

'Dans le Top 100 des films : US (72%) FR (10%)'

## Question n°7


In [64]:
liste_genre =[]
list_movies = movies.find()
for movie in list_movies:
    genre_movie = movie["genre"]
    for genre in genre_movie:
        if not genre in liste_genre:
            liste_genre.append(genre)
print(liste_genre, len(liste_genre))

['Drama', 'Adventure', 'Sci-Fi', 'Comedy', 'War', 'Action', 'Fantasy', 'Romance', 'Crime', 'Thriller', 'Mystery', 'Family', 'Biography', 'History', 'Animation', 'Film-Noir', 'Horror', 'Sport', 'Western', 'Music', 'Musical'] 21


In [65]:
dict_temps_genre={}
for genre in liste_genre:
    temps_total = 0
    list_movies = movies.find({"genre":{"$in":[genre]}})
    nb_movies=list_movies.count()
    
    for movie in list_movies:
        temps_total = movie["duree"] + temps_total

    dict_temps_genre[genre] =  temps_total/nb_movies



/tmp/ipykernel_18242/386818461.py:5: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  nb_movies=list_movies.count()


In [66]:
for key,value in dict_temps_genre.items():
    print(f"Le temps moyen du genre {key} est {round(value,2)} minutes ")

Le temps moyen du genre Drama est 134.64 minutes 
Le temps moyen du genre Adventure est 129.42 minutes 
Le temps moyen du genre Sci-Fi est 131.7 minutes 
Le temps moyen du genre Comedy est 106.33 minutes 
Le temps moyen du genre War est 140.43 minutes 
Le temps moyen du genre Action est 133.92 minutes 
Le temps moyen du genre Fantasy est 123.57 minutes 
Le temps moyen du genre Romance est 118.48 minutes 
Le temps moyen du genre Crime est 131.78 minutes 
Le temps moyen du genre Thriller est 121.03 minutes 
Le temps moyen du genre Mystery est 124.61 minutes 
Le temps moyen du genre Family est 114.54 minutes 
Le temps moyen du genre Biography est 147.07 minutes 
Le temps moyen du genre History est 156.0 minutes 
Le temps moyen du genre Animation est 100.96 minutes 
Le temps moyen du genre Film-Noir est 110.0 minutes 
Le temps moyen du genre Horror est 120.6 minutes 
Le temps moyen du genre Sport est 122.0 minutes 
Le temps moyen du genre Western est 149.33 minutes 
Le temps moyen du genre